# SyntheticMass Covid-19 Data Preperation

This notebook

- Joins our dataset into a one-event-per-visit-per-row arrangement
- Normalizes our data
- Applies Laplacian noise supporting Episolon-Differential Privacy
- Exports finalized data into training and test splits

In [28]:
import os
import numpy as np
import pydp as dp
from pydp.algorithms.laplacian import BoundedSum, BoundedMean, Count, Max
import pandas as pd

Read in all data

In [29]:
BASEPATH = "10k_synthea_covid19_csv"

conditions = pd.read_csv(os.path.join(BASEPATH, "conditions.csv"))     # Useful?
patients = pd.read_csv(os.path.join(BASEPATH, "patients.csv"))         # Not useful - Name, address, birthplace, bills, etc
observations = pd.read_csv(os.path.join(BASEPATH, "observations.csv")) # A single fact about a patient - patient XYZ cam in and weights 100 lbs. Patient ABC came in and said their pain was a 6/10. Patient DEF has a cranial measurement of 30", etc
care_plans = pd.read_csv(os.path.join(BASEPATH, "careplans.csv"))
devices = pd.read_csv(os.path.join(BASEPATH, "devices.csv"))           # Patient ABC uses a dialysis machine, etc
encounters = pd.read_csv(os.path.join(BASEPATH, "encounters.csv"))     # Not useful -  Patient visit justification with all relevant billing info
supplies = pd.read_csv(os.path.join(BASEPATH, "supplies.csv"))         # Not useful - equipment medical team used, like alcohol wipes or face masks
procedures = pd.read_csv(os.path.join(BASEPATH, "procedures.csv"))     # Procedure performed and associated billing 
medications = pd.read_csv(os.path.join(BASEPATH, "medications.csv"))   # MEDICATION PERSCRIBED

Grab the IDs of patients that have been diagnosed with COVID-19

In [30]:
covid_patient_ids = conditions[conditions.CODE == 840539006].PATIENT.unique()
negative_covid_patient_ids = observations[(observations.CODE == '94531-1') & (observations.VALUE == 'Not detected (qualifier value)')].PATIENT.unique()
deceased_patients = patients[patients.DEATHDATE.notna()].Id
completed_isolation_patients = care_plans[(care_plans.CODE == 736376001) & (care_plans.STOP.notna()) & (care_plans.REASONCODE == 840539006)].PATIENT
survivor_ids = np.union1d(completed_isolation_patients, negative_covid_patient_ids)
inpatient_ids = encounters[(encounters.REASONCODE == 840539006) & (encounters.CODE == 1505002)].PATIENT

In [31]:
lab_obs = observations[(observations.CODE == '48065-7') | (observations.CODE == '26881-3') | 
                          (observations.CODE == '2276-4') | (observations.CODE == '89579-7') |
                          (observations.CODE == '2532-0') | (observations.CODE == '731-0') |
                          (observations.CODE == '14804-9')
                      ]
covid_conditions = conditions[conditions.CODE == 840539006]
covid_patients = covid_conditions.merge(patients, how='left', left_on='PATIENT', right_on='Id')
covid_patients['SURVIVOR'] = covid_patients.PATIENT.isin(survivor_ids)

In [32]:
df1 = covid_patients[["PATIENT", "ENCOUNTER", "SURVIVOR"]]
results = pd.merge(df1, conditions[["ENCOUNTER", "CODE", "DESCRIPTION"]], on="ENCOUNTER", how="left")
results = pd.merge(results, medications[["ENCOUNTER", "CODE", "DESCRIPTION"]], on="ENCOUNTER", how="left")
results = pd.merge(results, devices[["ENCOUNTER", "CODE", "DESCRIPTION", "UDI"]], on="ENCOUNTER", how="left")
results

,PATIENT,ENCOUNTER,SURVIVOR,CODE_x,DESCRIPTION_x,CODE_y,DESCRIPTION_y,CODE,DESCRIPTION,UDI
0,f0f3bc8d-ef38-49ce-a2bd-dfdda982b271,681c380b-3c84-4c55-80a6-db3d9ea12fee,True,386661006,Fever (finding),NaN,NaN,NaN,NaN,NaN
1,f0f3bc8d-ef38-49ce-a2bd-dfdda982b271,681c380b-3c84-4c55-80a6-db3d9ea12fee,True,840544004,Suspected COVID-19,NaN,NaN,NaN,NaN,NaN
2,f0f3bc8d-ef38-49ce-a2bd-dfdda982b271,681c380b-3c84-4c55-80a6-db3d9ea12fee,True,840539006,COVID-19,NaN,NaN,NaN,NaN,NaN
3,067318a4-db8f-447f-8b6e-f2f61e9baaa5,1ea74a77-3ad3-4948-a9cc-3084462035d6,True,49727002,Cough (finding),NaN,NaN,NaN,NaN,NaN
4,067318a4-db8f-447f-8b6e-f2f61e9baaa5,1ea74a77-3ad3-4948-a9cc-3084462035d6,True,248595008,Sputum finding (finding),NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
53955,2712205f-755e-4897-acb3-926895b7d635,a18d0f16-bc12-4897-8844-8c3ebc2464fc,True,248595008,Sputum finding (finding),NaN,NaN,NaN,NaN,NaN
53956,2712205f-755e-4897-acb3-926895b7d635,a18d0f16-bc12-4897-8844-8c3ebc2464fc,True,68962001,Muscle pain (finding),NaN,NaN,NaN,NaN,NaN
53957,2712205f-755e-4897-acb3-926895b7d635,a18d0f16-bc12-4897-8844-8c3ebc2464fc,True,57676002,Joint pain (finding),NaN,NaN,NaN,NaN,NaN
53958,2712205f-755e-4897-acb3-926895b7d635,a18d0f16-bc12-4897-8844-8c3ebc2464fc,True,840544004,Suspected COVID-19,NaN,NaN,NaN,NaN,NaN


In [33]:
df1 = covid_patients[["PATIENT", "SURVIVOR"]]
cond_subset = conditions[["PATIENT", "ENCOUNTER", "CODE", "DESCRIPTION"]]
cond_subset.columns = ["PATIENT", "ENCOUNTER", "CONDITION_CODE", "CONDITION_DESCRIPTION"]
results = pd.merge(df1, cond_subset, on="PATIENT")

# We don't always have prescribed devices
dev_subset = devices[["ENCOUNTER", "CODE", "DESCRIPTION", "UDI"]]
dev_subset.columns = ["ENCOUNTER", "DEVICE_CODE", "DEVICE_DESCRIPTION", "UDI"]
results = pd.merge(results, dev_subset, on="ENCOUNTER", how="left")

# We also don't always have prescribed medications
med_subset = devices[["ENCOUNTER", "CODE", "DESCRIPTION"]]
med_subset.columns = ["ENCOUNTER", "MEDICATION_CODE", "MEDICATION_DESCRIPTION"]
results = pd.merge(results, med_subset, on="ENCOUNTER", how="left")

# We may have not given them a procedure
procedure_subset = procedures[["ENCOUNTER", "CODE", "DESCRIPTION"]]
procedure_subset.columns = ["ENCOUNTER", "PROCEDURE_CODE", "PROCEDURE_DESCRIPTION"]
results = pd.merge(results, procedure_subset, on="ENCOUNTER", how="left")

results

,PATIENT,SURVIVOR,ENCOUNTER,CONDITION_CODE,CONDITION_DESCRIPTION,DEVICE_CODE,DEVICE_DESCRIPTION,UDI,MEDICATION_CODE,MEDICATION_DESCRIPTION,PROCEDURE_CODE,PROCEDURE_DESCRIPTION
0,f0f3bc8d-ef38-49ce-a2bd-dfdda982b271,True,d5ee30a9-362f-429e-a87a-ee38d999b0a5,65363002,Otitis media,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,f0f3bc8d-ef38-49ce-a2bd-dfdda982b271,True,8bca6d8a-ab80-4cbf-8abb-46654235f227,65363002,Otitis media,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,f0f3bc8d-ef38-49ce-a2bd-dfdda982b271,True,681c380b-3c84-4c55-80a6-db3d9ea12fee,386661006,Fever (finding),NaN,NaN,NaN,NaN,NaN,261352009.0,Face mask (physical object)
3,f0f3bc8d-ef38-49ce-a2bd-dfdda982b271,True,681c380b-3c84-4c55-80a6-db3d9ea12fee,840544004,Suspected COVID-19,NaN,NaN,NaN,NaN,NaN,261352009.0,Face mask (physical object)
4,f0f3bc8d-ef38-49ce-a2bd-dfdda982b271,True,681c380b-3c84-4c55-80a6-db3d9ea12fee,840539006,COVID-19,NaN,NaN,NaN,NaN,NaN,261352009.0,Face mask (physical object)
...,...,...,...,...,...,...,...,...,...,...,...,...
1274596,2712205f-755e-4897-acb3-926895b7d635,True,a18d0f16-bc12-4897-8844-8c3ebc2464fc,248595008,Sputum finding (finding),NaN,NaN,NaN,NaN,NaN,261352009.0,Face mask (physical object)
1274597,2712205f-755e-4897-acb3-926895b7d635,True,a18d0f16-bc12-4897-8844-8c3ebc2464fc,68962001,Muscle pain (finding),NaN,NaN,NaN,NaN,NaN,261352009.0,Face mask (physical object)
1274598,2712205f-755e-4897-acb3-926895b7d635,True,a18d0f16-bc12-4897-8844-8c3ebc2464fc,57676002,Joint pain (finding),NaN,NaN,NaN,NaN,NaN,261352009.0,Face mask (physical object)
1274599,2712205f-755e-4897-acb3-926895b7d635,True,a18d0f16-bc12-4897-8844-8c3ebc2464fc,840544004,Suspected COVID-19,NaN,NaN,NaN,NaN,NaN,261352009.0,Face mask (physical object)


In [34]:
results = results[results['DEVICE_CODE'].notna()].reset_index(drop=True)

In [35]:
def perform_categorical_mapping(column_name):
    cond_counter = 0
    condition_mapping = {}
    for i, cond_code in enumerate(results[column_name].values):
        if cond_code not in condition_mapping:
            condition_mapping[cond_code] = cond_counter
            cond_counter += 1
        results[column_name].values[i] = condition_mapping[cond_code]
    return condition_mapping

In [36]:
condition_remapper = perform_categorical_mapping("CONDITION_CODE")
device_remapper = perform_categorical_mapping("DEVICE_CODE")
medication_remapper = perform_categorical_mapping("MEDICATION_CODE")
procedure_remapper = perform_categorical_mapping("PROCEDURE_CODE")

In [37]:
results['CONDITION_CODE'].fillna(-1, inplace=True)
results['DEVICE_CODE'].fillna(-1, inplace=True)
results['MEDICATION_CODE'].fillna(-1, inplace=True)
results['PROCEDURE_CODE'].fillna(-1, inplace=True)
results

,PATIENT,SURVIVOR,ENCOUNTER,CONDITION_CODE,CONDITION_DESCRIPTION,DEVICE_CODE,DEVICE_DESCRIPTION,UDI,MEDICATION_CODE,MEDICATION_DESCRIPTION,PROCEDURE_CODE,PROCEDURE_DESCRIPTION
0,0bccccae-0961-4ee0-896a-d80729b22e6c,True,87528e12-d377-4b70-adde-5f14126839f3,0,Cardiac Arrest,0.0,Implantable defibrillator device (physical ob...,(01)19898089248976(11)830326(17)080409(10)4261...,0.0,Implantable defibrillator device (physical ob...,0.0,NaN
1,0bccccae-0961-4ee0-896a-d80729b22e6c,True,87528e12-d377-4b70-adde-5f14126839f3,1,History of cardiac arrest (situation),0.0,Implantable defibrillator device (physical ob...,(01)19898089248976(11)830326(17)080409(10)4261...,0.0,Implantable defibrillator device (physical ob...,1.0,NaN
2,b837f045-b9b5-486a-ad32-94b9f519737f,True,a86dca9b-b318-445b-9ab6-d227e15b6021,0,Cardiac Arrest,0.0,Implantable defibrillator device (physical ob...,(01)32509593087287(11)860613(17)110628(10)6374...,0.0,Implantable defibrillator device (physical ob...,2.0,NaN
3,b837f045-b9b5-486a-ad32-94b9f519737f,True,a86dca9b-b318-445b-9ab6-d227e15b6021,1,History of cardiac arrest (situation),0.0,Implantable defibrillator device (physical ob...,(01)32509593087287(11)860613(17)110628(10)6374...,0.0,Implantable defibrillator device (physical ob...,3.0,NaN
4,c70992c9-ff13-467b-9032-1901506edeef,False,4f399fa3-79dc-4f0c-968d-e2c6247a3ed1,2,Septic shock (disorder),1.0,Mechanical ventilator (physical object),(01)51467824701974(11)200213(17)450227(10)6798...,1.0,Mechanical ventilator (physical object),4.0,Insertion of endotracheal tube (procedure)
...,...,...,...,...,...,...,...,...,...,...,...,...
1005696,cc08ece6-5a10-454f-b1d8-830a4b02fd31,False,fec4ecb2-f57d-44a1-afd5-d5c5f31ae3b2,10,Acute pulmonary embolism (disorder),3.0,Hemodialysis machine device (physical object),(01)33480890199181(11)200301(17)450316(10)6429...,3.0,Hemodialysis machine device (physical object),7.0,Hemodialysis (procedure)
1005697,cc08ece6-5a10-454f-b1d8-830a4b02fd31,False,fec4ecb2-f57d-44a1-afd5-d5c5f31ae3b2,10,Acute pulmonary embolism (disorder),3.0,Hemodialysis machine device (physical object),(01)33480890199181(11)200301(17)450316(10)6429...,3.0,Hemodialysis machine device (physical object),6.0,Controlled ventilation procedure and therapy ...
1005698,cc08ece6-5a10-454f-b1d8-830a4b02fd31,False,fec4ecb2-f57d-44a1-afd5-d5c5f31ae3b2,10,Acute pulmonary embolism (disorder),3.0,Hemodialysis machine device (physical object),(01)33480890199181(11)200301(17)450316(10)6429...,3.0,Hemodialysis machine device (physical object),7.0,Hemodialysis (procedure)
1005699,cc08ece6-5a10-454f-b1d8-830a4b02fd31,False,fec4ecb2-f57d-44a1-afd5-d5c5f31ae3b2,10,Acute pulmonary embolism (disorder),3.0,Hemodialysis machine device (physical object),(01)33480890199181(11)200301(17)450316(10)6429...,3.0,Hemodialysis machine device (physical object),6.0,Controlled ventilation procedure and therapy ...


In [38]:
def normalize(df):
    return (df-df.min())/(df.max()-df.min())

In [80]:
def apply_diffprivlib_to_row(row):
    # Example diffprivlib operation - Laplace mechanism
    epsilon = 0.5
    sensitivity = 1  # Assuming the sensitivity of each entry is 1

    # Create a Laplace mechanism
    laplace = Laplace(epsilon=epsilon, sensitivity=sensitivity)
    
    # Apply diffprivlib operation to each value in the row
    result = [value + laplace.randomise(value) for value in row if isinstance(value, (int, float, complex)) and not isinstance(value, str)]
    
    return result

In [81]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from diffprivlib.mechanisms import Laplace

n = 10000
X = results[:n]
Xk = results[['CONDITION_CODE', 'PROCEDURE_CODE']][:n]
y = results['DEVICE_CODE'][:n]
df = Xk.apply(apply_diffprivlib_to_row, axis=1)
c1=[]
c2=[]
for pair in df:
    try:
        c1.append(pair[0])
        c2.append(pair[1])
    except Exception:
        print(pair)

Xk = Xk.assign(CONDITION_CODE=c1)
Xk = Xk.assign(PROCEDURE_CODE=c2)

Xk = normalize(Xk)

X_train, X_test, y_train, y_test = train_test_split(Xk, y, test_size=0.33, random_state=42)

X_train.to_csv("X.txt", sep=' ', index=False, header=False)
X_test.to_csv("Xeval.txt", sep=' ', index=False, header=False)
y_train.to_csv("y.txt", sep=' ', index=False, header=False)
y_test.to_csv("yeval.txt", sep=' ', index=False, header=False)

In [61]:
Xk

,CONDITION_CODE,PROCEDURE_CODE
0,0.292987,0.252909
1,0.405659,0.321519
2,0.305971,0.311696
3,0.377147,0.315260
4,0.451253,0.409949
...,...,...
9995,0.677504,0.320620
9996,0.687766,0.518055
9997,0.569341,0.520335
9998,0.591732,0.485505


In [64]:
y

0       0.0
1       0.0
2       0.0
3       0.0
4       1.0
       ... 
9995    3.0
9996    3.0
9997    3.0
9998    3.0
9999    3.0
Name: DEVICE_CODE, Length: 10000, dtype: float64